In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import os
import watershed_udder as wu
from PIL import Image
from skimage import exposure
import pandas as pd
import watershed_udder as wu

In [22]:
color_dict = {"lf":[255,255,0], "rf": [0, 255, 255], "lb":[255, 0,255], "rb":[255,0,0], "bg": [0, 0, 0]}

In [23]:
def mk_dir(dirpath):
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)

rc = {"axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : False,
      "xtick.labelbottom" : False,
      "ytick.labelleft" : False,
      "ytick.left" : False}

plt.rcParams.update(rc)

dirpath = os.getcwd()
label_dir = r"validate_watershed\watershed_segments"
corr_dir = r"validate_watershed\watershed_correspondence"
im_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r"udder_video\depth_images")
out_dir = r"validate_watershed\watershed_data"
# file_list = os.listdir(label_dir)
df = pd.read_csv(r"validate_watershed\survey_groups.csv")
file_list = [file for file in df.filename]


cnt = 0
for file in file_list:
    label_file = os.path.join(label_dir, file+".npy")
    img_file = file + ".tif"
    cow = file.split("_")[0]
    
    img = Image.open(os.path.join(im_dir, cow, img_file))
    labels = np.load(label_file)
    labels_mask =labels.copy().astype("float")
    img2 = exposure.equalize_hist(np.asarray(img), nbins=256, mask=None)
    labels_mask =labels.copy().astype("float")
    labels_mask[labels_mask==0] = np.nan
    kp_ws = pd.read_csv(os.path.join(corr_dir, file +".csv")).loc[0].to_dict()
    ws_map = dict((v, k) for k, v in kp_ws.items())
    ws_map[0] = "bg"
    labels_img = np.zeros((labels.shape[0], labels.shape[1], 3))
    for key in ws_map.keys():
        quarter_mask = labels.copy()
        quarter_mask[labels !=key] = 0
        rows,cols = np.nonzero(quarter_mask)
        labels_img[rows, cols, :] = color_dict[ws_map[key]]
    im = Image.fromarray(np.uint8(labels_img))
    im.save(os.path.join(out_dir,"ws_mask", file + ".png"))
    fig, ax = plt.subplots(frameon = False)
    ax.imshow(img2, cmap='gray')
    ax.imshow(labels_img/255, alpha=0.3)
    fig.savefig(os.path.join(out_dir, "masked_frame", file + ".png"), transparent = True)
    plt.close(fig)
    
    print(f"{cnt}: {file}")
    cnt +=1

0: 1057_20231117_140050_frame_621
1: 1057_20231117_140050_frame_286
2: 1057_20231117_140050_frame_611
3: 1057_20231117_140050_frame_610
4: 1057_20231117_140050_frame_629
5: 1057_20231117_140050_frame_633
6: 970_20231117_150812_frame_629
7: 970_20231117_150812_frame_622
8: 970_20231117_150812_frame_222
9: 970_20231117_150812_frame_551
10: 970_20231117_150812_frame_632
11: 970_20231117_150812_frame_613
12: 764_20231117_144922_frame_812
13: 764_20231117_144922_frame_696
14: 764_20231117_144922_frame_769
15: 764_20231117_144922_frame_843
16: 764_20231117_144922_frame_809
17: 764_20231117_144922_frame_689
18: 1223_20231117_110830_frame_760
19: 1223_20231117_110830_frame_551
20: 1223_20231117_110830_frame_639
21: 1223_20231117_110830_frame_785
22: 1223_20231117_110830_frame_745
23: 1223_20231117_110830_frame_536
24: 1116_20231117_141845_frame_248
25: 1116_20231117_141845_frame_245
26: 1116_20231117_141845_frame_274
27: 1116_20231117_141845_frame_278
28: 1116_20231117_141845_frame_916
29: 111